In [0]:
import dlt
import sys
from pyspark.sql.functions import col
sys.path.append("/Workspace/Project_1/Databricks_Projects")
from validation_rules import not_null_rule, date_format_rule, concat_errors

def validation_columns(df):
    invalid_customer_name = not_null_rule("customer_name")
    invalid_date_format = date_format_rule("date")
    invalid_date_null = not_null_rule("date")
    Invalid_summary = concat_errors(invalid_customer_name, invalid_date_format, invalid_date_null)
    df_validated = (
        df
        .withColumn("invalid_customer_name", invalid_customer_name)
        .withColumn("invalid_date_format", invalid_date_format)
        .withColumn("invalid_date_null", invalid_date_null)
        .withColumn("Invalid_summary", Invalid_summary)
    )
    return df_validated

@dlt.table(
    name="vld_ar_receipt_valid"
)
def vld_ar_receipt_valid():
    df = dlt.read("cln_ar_receipt_cleanse")
    df_validated = validation_columns(df)
    return df_validated.filter(col("Invalid_summary") == "")

@dlt.table(
    name="vld_ar_receipt_invalid"
)
def vld_ar_receipt_invalid():
    df = dlt.read("cln_ar_receipt_cleanse")
    df_validated = validation_columns(df)
    return df_validated.filter(col("Invalid_summary") != "")